In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical

In [2]:
df= pd.read_csv('clean_data.csv')
df

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,112641719,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,112641466,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,112638623,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,6,6453966,15,10,1239,2273,744,0,82.600000,196.741237,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,6,8804066,14,11,1143,2209,744,0,81.642857,203.745545,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,6,19276110,5,10,286,738,103,0,57.200000,52.485236,...,20,9667037.0,0.0,9667037,9667037,9609073.0,0.000000,9609073,9609073,Benign
2798300,6,85992425,2,0,0,0,0,0,0.000000,0.000000,...,20,0.0,0.0,0,0,85992425.0,0.000000,85992425,85992425,Benign
2798301,17,79134,1,1,41,98,41,41,41.000000,0.000000,...,8,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
2798302,6,10053785,11,12,1321,1659,677,0,120.090909,197.116440,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798304 entries, 0 to 2798303
Data columns (total 70 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Protocol           int64  
 1   Flow Duration      int64  
 2   Tot Fwd Pkts       int64  
 3   Tot Bwd Pkts       int64  
 4   TotLen Fwd Pkts    int64  
 5   TotLen Bwd Pkts    int64  
 6   Fwd Pkt Len Max    int64  
 7   Fwd Pkt Len Min    int64  
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    int64  
 11  Bwd Pkt Len Min    int64  
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float64
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT Mean      float64
 17  Flow IAT Std       float64
 18  Flow IAT Max       int64  
 19  Flow IAT Min       int64  
 20  Fwd IAT Tot        int64  
 21  Fwd IAT Mean       float64
 22  Fwd IAT Std        float64
 23  Fwd IAT Max        int64  
 24  Fwd IAT Min        int64  
 25  Bwd IAT Tot       

In [4]:
df.columns

Index(['Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Subflow Fwd Pkts'

In [5]:
df['Label'].unique()

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce', 'Infilteration',
       'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection'],
      dtype=object)

In [6]:
df['Label'].value_counts()

Benign              2589517
Infilteration        113819
SSH-Bruteforce        94048
Brute Force -Web        555
Brute Force -XSS        228
SQL Injection            84
FTP-BruteForce           53
Name: Label, dtype: int64

In [7]:
Label = pd.DataFrame(df.Label)
Label

,Label
0,Benign
1,Benign
2,Benign
3,Benign
4,Benign
...,...
2798299,Benign
2798300,Benign
2798301,Benign
2798302,Benign


In [8]:
std_scaler = MinMaxScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = df.select_dtypes(include='number').columns
df = standardization(df,numeric_col)

<IPython.core.display.Javascript object>

In [9]:
df

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000000,0.938681,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.469340,0.000002,0.469341,0.469340,Benign
1,0.000000,0.938679,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.469339,0.000002,0.469340,0.469339,Benign
2,0.000000,0.938655,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.469328,0.000004,0.469329,0.469326,Benign
3,0.352941,0.053783,0.000115,0.000081,8.580817e-06,1.453693e-05,0.011546,0.000000,0.007364,0.012483,...,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign
4,0.352941,0.073367,0.000107,0.000089,7.915959e-06,1.412762e-05,0.011546,0.000000,0.007278,0.012927,...,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,0.352941,0.160634,0.000033,0.000081,1.980721e-06,4.719864e-06,0.001598,0.000000,0.005099,0.003330,...,0.416667,0.084799,0.0,0.084799,0.084799,0.080076,0.000000,0.080076,0.080076,Benign
2798300,0.352941,0.716604,0.000008,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.716604,0.000000,0.716604,0.716604,Benign
2798301,1.000000,0.000659,0.000000,0.000008,2.839496e-07,6.267571e-07,0.000636,0.028082,0.003655,0.000000,...,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign
2798302,0.352941,0.083782,0.000082,0.000097,9.148716e-06,1.061010e-05,0.010506,0.000000,0.010706,0.012507,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798304 entries, 0 to 2798303
Data columns (total 70 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Protocol           float64
 1   Flow Duration      float64
 2   Tot Fwd Pkts       float64
 3   Tot Bwd Pkts       float64
 4   TotLen Fwd Pkts    float64
 5   TotLen Bwd Pkts    float64
 6   Fwd Pkt Len Max    float64
 7   Fwd Pkt Len Min    float64
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    float64
 11  Bwd Pkt Len Min    float64
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float64
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT Mean      float64
 17  Flow IAT Std       float64
 18  Flow IAT Max       float64
 19  Flow IAT Min       float64
 20  Fwd IAT Tot        float64
 21  Fwd IAT Mean       float64
 22  Fwd IAT Std        float64
 23  Fwd IAT Max        float64
 24  Fwd IAT Min        float64
 25  Bwd IAT Tot       

In [11]:
X = df.drop('Label', axis=1)
y = df['Label']

In [12]:
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
y_encoded = pd.DataFrame(y_encoded)
y_encoded

C:\Users\aryan\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2798299,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798300,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798301,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798302,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
X 

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000000,0.938681,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.469340,0.000002,0.469341,0.469340
1,0.000000,0.938679,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.469339,0.000002,0.469340,0.469339
2,0.000000,0.938655,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.469328,0.000004,0.469329,0.469326
3,0.352941,0.053783,0.000115,0.000081,8.580817e-06,1.453693e-05,0.011546,0.000000,0.007364,0.012483,...,0.000049,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.352941,0.073367,0.000107,0.000089,7.915959e-06,1.412762e-05,0.011546,0.000000,0.007278,0.012927,...,0.000049,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,0.352941,0.160634,0.000033,0.000081,1.980721e-06,4.719864e-06,0.001598,0.000000,0.005099,0.003330,...,0.000025,0.416667,0.084799,0.0,0.084799,0.084799,0.080076,0.000000,0.080076,0.080076
2798300,0.352941,0.716604,0.000008,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.416667,0.000000,0.0,0.000000,0.000000,0.716604,0.000000,0.716604,0.716604
2798301,1.000000,0.000659,0.000000,0.000008,2.839496e-07,6.267571e-07,0.000636,0.028082,0.003655,0.000000,...,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2798302,0.352941,0.083782,0.000082,0.000097,9.148716e-06,1.061010e-05,0.010506,0.000000,0.010706,0.012507,...,0.000058,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
df = pd.concat([X, y_encoded], axis=1)
df

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Idle Std,Idle Max,Idle Min,0,1,2,3,4,5,6
0,0.000000,0.938681,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000002,0.469341,0.469340,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.938679,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000002,0.469340,0.469339,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.938655,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000004,0.469329,0.469326,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.352941,0.053783,0.000115,0.000081,8.580817e-06,1.453693e-05,0.011546,0.000000,0.007364,0.012483,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.352941,0.073367,0.000107,0.000089,7.915959e-06,1.412762e-05,0.011546,0.000000,0.007278,0.012927,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,0.352941,0.160634,0.000033,0.000081,1.980721e-06,4.719864e-06,0.001598,0.000000,0.005099,0.003330,...,0.000000,0.080076,0.080076,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798300,0.352941,0.716604,0.000008,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.716604,0.716604,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798301,1.000000,0.000659,0.000000,0.000008,2.839496e-07,6.267571e-07,0.000636,0.028082,0.003655,0.000000,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798302,0.352941,0.083782,0.000082,0.000097,9.148716e-06,1.061010e-05,0.010506,0.000000,0.010706,0.012507,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798304 entries, 0 to 2798303
Data columns (total 76 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Protocol           float64
 1   Flow Duration      float64
 2   Tot Fwd Pkts       float64
 3   Tot Bwd Pkts       float64
 4   TotLen Fwd Pkts    float64
 5   TotLen Bwd Pkts    float64
 6   Fwd Pkt Len Max    float64
 7   Fwd Pkt Len Min    float64
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    float64
 11  Bwd Pkt Len Min    float64
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float64
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT Mean      float64
 17  Flow IAT Std       float64
 18  Flow IAT Max       float64
 19  Flow IAT Min       float64
 20  Fwd IAT Tot        float64
 21  Fwd IAT Mean       float64
 22  Fwd IAT Std        float64
 23  Fwd IAT Max        float64
 24  Fwd IAT Min        float64
 25  Bwd IAT Tot       

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2)
print(X_train.shape," ",X_test.shape)
print(y_train.shape," ",y_test.shape)

(2238643, 69)   (559661, 69)
(2238643, 7)   (559661, 7)


In [17]:
#selecting 1% of random rows for better running time

X_train = X_train.sample(frac=0.1, replace=True, random_state=1)
y_train = y_train.sample(frac=0.1, replace=True, random_state=1)
X_test = X_test.sample(frac=0.1, replace=True, random_state=1)
y_test = y_test.sample(frac=0.1, replace=True, random_state=1)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(223864, 69) (223864, 7)
(55966, 69) (55966, 7)


In [19]:
from sklearn.feature_selection import SelectFromModel

sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1))
sel.fit(X_train, y_train)
sel.get_support()

X_train.columns

features = X_train.columns[sel.get_support()]
print(features)

print(len(features))

np.mean(sel.estimator_.feature_importances_)
print(sel.estimator_.feature_importances_)

X_train_rfe = sel.transform(X_train)
X_test_rfe = sel.transform(X_test)

<IPython.core.display.Javascript object>

Index(['Flow Duration', 'Tot Fwd Pkts', 'Flow Byts/s', 'Flow Pkts/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Mean',
       'Pkt Len Var', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts',
       'Subflow Bwd Pkts', 'Init Fwd Win Byts', 'Fwd Act Data Pkts',
       'Fwd Seg Size Min'],
      dtype='object')
27
[1.28642775e-04 3.25484401e-02 1.90830265e-02 1.44420416e-02
 6.92568689e-03 1.37374953e-02 1.29198986e-02 3.10162237e-03
 7.05567255e-03 4.27135196e-03 6.22562631e-03 3.91538148e-03
 1.05701110e-02 4.24021375e-03 1.88635494e-02 2.61532582e-02
 2.78058975e-02 2.27471711e-02 3.56789997e-02 1.51053940e-02
 3.07743987e-02 2.56918201e-02 1.48980658e-02 2.20274970e-02
 1.18366917e-02 1.46644679e-02 1.60028720e-02 1.48778811e-02
 1.20228289e-02 9.21160984e-03 2.231435

In [20]:
X_train_rfe=np.array(X_train_rfe)
X_test_rfe=np.array(X_test_rfe)
y=np.array(y)

In [21]:
X_train = np.reshape(X_train_rfe, ( X_train_rfe.shape[0], 1 , X_train_rfe.shape[1] ))
X_test = np.reshape(X_test_rfe, ( X_test_rfe.shape[0], 1,  X_test_rfe.shape[1] ))

In [22]:
from keras.layers import Dense, LSTM, MaxPool1D, Flatten, Dropout # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.layers import Input
from keras.models import Model

model = Sequential() # initializing model
model.add(LSTM(64,return_sequences=True,input_shape = (1, X_train_rfe.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Flatten())
model.add(Dense(units=50))
# output layer with softmax activation
model.add(Dense(units=7,activation='softmax'))

In [23]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# summary of model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             23552     
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 64)             33024     
                                                                 
 dropout_1 (Dropout)         (None, 1, 64)             0         
                                                                 
 lstm_2 (LSTM)               (None, 1, 64)             33024     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 50)                3

In [25]:
history = model.fit(X_train, y_train, epochs=100, batch_size=5000,validation_split=0.2)

Epoch 1/100


36/36 [==============================] - 7s 78ms/step - loss: 1.5378 - accuracy: 0.8992 - val_loss: 0.7228 - val_accuracy: 0.9265
Epoch 2/100
36/36 [==============================] - 2s 49ms/step - loss: 0.4502 - accuracy: 0.9250 - val_loss: 0.3647 - val_accuracy: 0.9265
Epoch 3/100
36/36 [==============================] - 2s 48ms/step - loss: 0.3534 - accuracy: 0.9250 - val_loss: 0.3310 - val_accuracy: 0.9265
Epoch 4/100
36/36 [==============================] - 2s 48ms/step - loss: 0.3283 - accuracy: 0.9250 - val_loss: 0.3146 - val_accuracy: 0.9265
Epoch 5/100
36/36 [==============================] - 2s 54ms/step - loss: 0.3141 - accuracy: 0.9250 - val_loss: 0.3025 - val_accuracy: 0.9265
Epoch 6/100
36/36 [==============================] - 3s 84ms/step - loss: 0.3021 - accuracy: 0.9250 - val_loss: 0.2905 - val_accuracy: 0.9265
Epoch 7/100
36/36 [==============================] - 3s 80ms/step - loss: 0.2909 - accuracy: 0.9250 - val_loss: 0.2810 - val_accuracy: 0.9265
Epoc

36/36 [==============================] - 3s 73ms/step - loss: 0.1741 - accuracy: 0.9571 - val_loss: 0.1690 - val_accuracy: 0.9583
Epoch 56/100
36/36 [==============================] - 3s 76ms/step - loss: 0.1739 - accuracy: 0.9571 - val_loss: 0.1689 - val_accuracy: 0.9583
Epoch 57/100
36/36 [==============================] - 3s 81ms/step - loss: 0.1736 - accuracy: 0.9572 - val_loss: 0.1689 - val_accuracy: 0.9582
Epoch 58/100
36/36 [==============================] - 3s 76ms/step - loss: 0.1736 - accuracy: 0.9573 - val_loss: 0.1686 - val_accuracy: 0.9583
Epoch 59/100
36/36 [==============================] - 3s 79ms/step - loss: 0.1735 - accuracy: 0.9572 - val_loss: 0.1694 - val_accuracy: 0.9583
Epoch 60/100
36/36 [==============================] - 3s 76ms/step - loss: 0.1733 - accuracy: 0.9572 - val_loss: 0.1685 - val_accuracy: 0.9583
Epoch 61/100
36/36 [==============================] - 3s 75ms/step - loss: 0.1733 - accuracy: 0.9573 - val_loss: 0.1689 - val_accuracy: 0.9583
Epoch 62/100

In [26]:
# predicting target attribute on testing dataset
test_results = model.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

1749/1749 [==============================] - 3s 2ms/step - loss: 0.1685 - accuracy: 0.9575
Test results - Loss: 0.16853511333465576 - Accuracy: 95.75456380844116%
